# Web API Tutorial with gdmongolite

Learn how to create professional REST APIs in minutes with gdmongolite and FastAPI integration.

## What you'll learn:
- Create REST API with one line of code
- Auto-generated API documentation
- Custom endpoints and business logic
- Authentication and security
- Production deployment

## Step 1: Setup and Imports

In [ ]:
from gdmongolite import DB, Schema, Email, FieldTypes, create_fastapi_app
from fastapi import FastAPI, HTTPException
import uvicorn

print("Imports successful!")

## Step 2: Define Data Models

Let's create models for a simple e-commerce API:

In [ ]:
# Initialize database
db = DB()

class User(Schema):
    """User model"""
    name: FieldTypes.Name
    email: Email
    age: FieldTypes.Age
    role: str = "user"  # user, admin, moderator
    is_active: bool = True

class Product(Schema):
    """Product model"""
    name: FieldTypes.Title
    description: FieldTypes.Description
    price: FieldTypes.Price
    category: str
    in_stock: bool = True
    stock_quantity: int = 0

class Order(Schema):
    """Order model"""
    user_id: str
    product_ids: list[str]
    total_amount: FieldTypes.Price
    status: str = "pending"  # pending, confirmed, shipped, delivered
    created_at: str = None

# Register all schemas
for schema in [User, Product, Order]:
    db.register_schema(schema)

print("Models defined and registered!")

## Step 3: Create FastAPI App (One Line!)

This automatically creates all CRUD endpoints for your models:

In [ ]:
# Create FastAPI app with auto-generated CRUD routes
app = create_fastapi_app(
    db,
    schemas=[User, Product, Order],
    title="E-commerce API",
    description="Professional e-commerce API built with gdmongolite",
    version="1.0.0"
)

print("FastAPI app created with auto-generated endpoints!")
print("\nAuto-generated endpoints:")
print("- GET /users/ - List users with pagination")
print("- POST /users/ - Create new user")
print("- GET /users/{id} - Get specific user")
print("- PUT /users/{id} - Update user")
print("- DELETE /users/{id} - Delete user")
print("- POST /users/search - Advanced search")
print("\nSame endpoints for Products and Orders!")

## Step 4: Add Custom Endpoints

Let's add some business logic with custom endpoints:

In [ ]:
@app.get("/")
async def welcome():
    """Welcome message"""
    return {
        "message": "Welcome to E-commerce API!",
        "docs": "/docs",
        "health": "/health",
        "version": "1.0.0"
    }

@app.get("/users/stats")
async def user_statistics():
    """Get user statistics"""
    total = await db.User.find().count()
    active = await db.User.find(is_active=True).count()
    admins = await db.User.find(role="admin").count()
    
    return {
        "total_users": total,
        "active_users": active,
        "admin_users": admins,
        "inactive_users": total - active
    }

@app.get("/products/categories")
async def product_categories():
    """Get all product categories"""
    categories = await db.Product.find().distinct("category")
    return {"categories": categories}

@app.get("/products/low-stock")
async def low_stock_products(threshold: int = 10):
    """Get products with low stock"""
    products = await db.Product.find(
        stock_quantity__lte=threshold,
        in_stock=True
    ).to_list()
    
    return {
        "low_stock_products": products,
        "count": len(products),
        "threshold": threshold
    }

@app.post("/orders/calculate")
async def calculate_order_total(product_ids: list[str]):
    """Calculate order total from product IDs"""
    total = 0
    products = []
    
    for product_id in product_ids:
        from bson import ObjectId
        try:
            product = await db.Product.find(_id=ObjectId(product_id)).first()
            if product:
                total += product["price"]
                products.append({
                    "id": str(product["_id"]),
                    "name": product["name"],
                    "price": product["price"]
                })
        except:
            continue
    
    return {
        "products": products,
        "total_amount": total,
        "currency": "USD",
        "item_count": len(products)
    }

print("Custom endpoints added!")

## Step 5: Add Sample Data

Let's add some sample data to test our API:

In [ ]:
# Add sample users
sample_users = [
    {
        "name": "John Admin",
        "email": "admin@example.com",
        "age": 35,
        "role": "admin"
    },
    {
        "name": "Jane Customer",
        "email": "jane@example.com",
        "age": 28,
        "role": "user"
    },
    {
        "name": "Bob Moderator",
        "email": "bob@example.com",
        "age": 32,
        "role": "moderator"
    }
]

user_response = await db.User.insert(sample_users)
print(f"Added {user_response.count} sample users")

# Add sample products
sample_products = [
    {
        "name": "Laptop Pro",
        "description": "High-performance laptop for professionals",
        "price": 1299.99,
        "category": "Electronics",
        "stock_quantity": 50
    },
    {
        "name": "Wireless Mouse",
        "description": "Ergonomic wireless mouse",
        "price": 29.99,
        "category": "Electronics",
        "stock_quantity": 5  # Low stock!
    },
    {
        "name": "Coffee Mug",
        "description": "Ceramic coffee mug",
        "price": 12.99,
        "category": "Home",
        "stock_quantity": 100
    }
]

product_response = await db.Product.insert(sample_products)
print(f"Added {product_response.count} sample products")

## Step 6: Test API Endpoints

Let's test our custom endpoints:

In [ ]:
# Test user statistics endpoint
stats = await user_statistics()
print("User Statistics:")
print(f"  Total users: {stats['total_users']}")
print(f"  Active users: {stats['active_users']}")
print(f"  Admin users: {stats['admin_users']}")

In [ ]:
# Test product categories endpoint
categories = await product_categories()
print(f"Product Categories: {categories['categories']}")

In [ ]:
# Test low stock products endpoint
low_stock = await low_stock_products(threshold=10)
print(f"Low Stock Products ({low_stock['count']} items):")
for product in low_stock['low_stock_products']:
    print(f"  - {product['name']}: {product['stock_quantity']} left")

## Step 7: Advanced Features

Let's add some advanced features like authentication and middleware:

In [ ]:
from fastapi import Depends, HTTPException, status
from fastapi.security import HTTPBearer, HTTPAuthorizationCredentials

# Simple authentication (in production, use proper JWT)
security = HTTPBearer()

async def get_current_user(credentials: HTTPAuthorizationCredentials = Depends(security)):
    """Get current user from token"""
    token = credentials.credentials
    
    # In production, validate JWT token here
    # For demo, we'll use email as token
    user = await db.User.find(email=token).first()
    
    if not user:
        raise HTTPException(
            status_code=status.HTTP_401_UNAUTHORIZED,
            detail="Invalid authentication credentials",
            headers={"WWW-Authenticate": "Bearer"},
        )
    
    return user

async def require_admin(current_user: dict = Depends(get_current_user)):
    """Require admin role"""
    if current_user["role"] != "admin":
        raise HTTPException(
            status_code=status.HTTP_403_FORBIDDEN,
            detail="Admin access required"
        )
    return current_user

# Protected endpoint
@app.get("/profile")
async def get_profile(current_user: dict = Depends(get_current_user)):
    """Get current user profile"""
    return {
        "user": current_user,
        "message": f"Hello {current_user['name']}!"
    }

# Admin-only endpoint
@app.get("/admin/dashboard")
async def admin_dashboard(admin_user: dict = Depends(require_admin)):
    """Admin dashboard with system stats"""
    user_count = await db.User.find().count()
    product_count = await db.Product.find().count()
    order_count = await db.Order.find().count()
    
    return {
        "admin": admin_user["name"],
        "stats": {
            "users": user_count,
            "products": product_count,
            "orders": order_count
        }
    }

print("Authentication and authorization added!")

## Step 8: Run the Server

Now let's start the server (uncomment to run):

In [ ]:
# Uncomment to run the server
# This will start the server on http://localhost:8000

print("To run the server, uncomment the code below:")
print("")
print("if __name__ == '__main__':")
print("    uvicorn.run(app, host='0.0.0.0', port=8000)")
print("")
print("Then visit:")
print("- http://localhost:8000 - Welcome page")
print("- http://localhost:8000/docs - Interactive API docs")
print("- http://localhost:8000/health - Health check")
print("- http://localhost:8000/users/stats - User statistics")

# if __name__ == '__main__':
#     uvicorn.run(app, host='0.0.0.0', port=8000)

## Step 9: Testing the API

Here's how to test your API endpoints:

In [ ]:
# Example API calls (using requests library)
import json

print("Example API calls:")
print("")
print("# Get all users")
print("GET http://localhost:8000/users/")
print("")
print("# Create new user")
print("POST http://localhost:8000/users/")
print(json.dumps({
    "name": "New User",
    "email": "newuser@example.com",
    "age": 25
}, indent=2))
print("")
print("# Search users")
print("POST http://localhost:8000/users/search")
print(json.dumps({
    "age__gte": 25,
    "role": "user"
}, indent=2))
print("")
print("# Get user profile (with auth)")
print("GET http://localhost:8000/profile")
print("Authorization: Bearer admin@example.com")

## Step 10: Production Deployment

Here's how to deploy your API to production:

In [ ]:
# Production configuration
from gdmongolite import prod_serve

print("Production deployment options:")
print("")
print("1. Using gdmongolite prod_serve:")
print("   prod_serve(db, [User, Product, Order], workers=4)")
print("")
print("2. Using Gunicorn:")
print("   gunicorn -w 4 -k uvicorn.workers.UvicornWorker app:app")
print("")
print("3. Using Docker:")
print("   FROM python:3.11")
print("   COPY . /app")
print("   WORKDIR /app")
print("   RUN pip install gdmongolite")
print("   CMD ['uvicorn', 'app:app', '--host', '0.0.0.0', '--port', '8000']")
print("")
print("4. Environment variables for production:")
print("   MONGO_URI=mongodb://prod-server:27017")
print("   MONGO_DB=production")
print("   GDMONGO_DEBUG=false")
print("   GDMONGO_LOG_QUERIES=false")

## Congratulations!

You've just created a professional REST API with:

- Auto-generated CRUD endpoints for all models
- Custom business logic endpoints
- Authentication and authorization
- Interactive API documentation
- Error handling and validation
- Production-ready configuration

## What you get automatically:

- **OpenAPI/Swagger docs** at `/docs`
- **Health check** endpoint at `/health`
- **CORS** support for web frontends
- **Automatic validation** of request/response data
- **Error handling** with proper HTTP status codes
- **Pagination** for list endpoints
- **Filtering and sorting** capabilities
- **Database statistics** at `/stats`

## Next Steps:

1. Add more custom endpoints for your business logic
2. Implement proper JWT authentication
3. Add rate limiting and caching
4. Set up monitoring and logging
5. Deploy to your favorite cloud platform

You're now ready to build production-grade APIs with gdmongolite!